# Assignment 1 - Ling 380

| Name (Last,First) | Student ID | Section Contributed | Section Edited | Other Contributions |
| --- | --- | --- | --- | --- |
| Bamba, Anant | 301430478 | Data collection, Github setup, code | section 3 and 4 | Section 5 writeup
| Bhullar, Harmeet | 301541206 | Data collection, Code | Section 1 and 2 | Initialising directories


In [56]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
import string
import pandas as pd
import csv

nltk.download('punkt')
# import the NLTK packages we know we need

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# these 2 packages may already be in your system, but just in case

nltk.download('wordnet')

# import spaCy and the small English language model
import spacy
nlp = spacy.load("en_core_web_sm")

# this does prettier displays on spaCy
from spacy import displacy

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


# Section 1 AND 2 - Length & Lexical Diversity 

In [49]:
def analyze_texts(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):  
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()  # Read the text from the file
                
                # Tokenization & Lexical Diversity
                tokens = word_tokenize(text)
                unique_tokens = set(tokens)
                lexical_diversity = len(unique_tokens) / len(tokens)

                print(f"File: {filename}")
                print(f"Length (tokens): {len(tokens)}")
                print(f"Lexical diversity: {lexical_diversity:.4f}")

#Directory path
data_directory = 'A1'
analyze_texts(data_directory)

File: OscarsNews.txt
Length (tokens): 6157
Lexical diversity: 0.2896
File: Script.txt
Length (tokens): 23538
Lexical diversity: 0.1339
File: recipes.txt
Length (tokens): 10670
Lexical diversity: 0.1619


# Section 3 - FreqDist

In [60]:


# import the library
import os

# define the path
# I'm trying to always use "data" as the place to store texts
path = '/Users/anantbamba/Documents/Jupyter/data/A1'

tokens_files = {}
top_10_words ={}
specific_puncs = ['``', '’', "n't", "'s",'“','”',"''"]
# create the list of things we'll remove (some punctuation, stopwords, and end of line codes)

stop_words = set(stopwords.words('english'))
my_punctuation = string.punctuation +'"'+'"'+'-'+'''+'''+'—'
removal_list = set(stop_words) | set(my_punctuation) | {'lt','rt', '\n'}
# loop through all the files in the directory "data"
for filename in os.listdir(path):
    # check only for .txt files
    if filename.endswith(".txt"):
        
        # get all the filenames with a .txt extension
        file_path = os.path.join(path, filename)
        
        # open one file at a time, to read it, and with utf encoding
        with open(file_path, 'r', encoding="utf-8") as f:
            # store the contents of the file into the variable "text"
            text = f.read()
            # tokenize the text using NLTK
            tokens = nltk.word_tokenize(text)
            # store the length of the variable tokens into tokens_files
            tokens_files[filename] = len(tokens)
            fdist = FreqDist(tokens)
            punctuations = list(string.punctuation)
            fdist = FreqDist([token for token in fdist if token not in punctuations])
            #getting rid of specific words and punctuations 
            fdist = FreqDist([token for token in fdist if token not in specific_puncs])
            fdist = FreqDist([token for token in fdist if token not in removal_list])
            top_10_word = fdist.most_common(10)
            top_10_words[filename] = top_10_word


In [61]:
top_10_words

{'recipes.txt': [('recipe', 1),
  ('I', 1),
  ('chicken', 1),
  ('pasta', 1),
  ('Chicken', 1),
  ('shrimp', 1),
  ('This', 1),
  ('make', 1),
  ('minutes', 1),
  ('If', 1)],
 'OscarsNews.txt': [('The', 1),
  ('I', 1),
  ('Oscars', 1),
  ('film', 1),
  ('best', 1),
  ('said', 1),
  ('show', 1),
  ('Wicked', 1),
  ('It', 1),
  ('year', 1)],
 'Script.txt': [('I', 1),
  ('JACOB', 1),
  ('CONTINUED', 1),
  ('Jacob', 1),
  ('Malee', 1),
  ('Leonard', 1),
  ('MALEE', 1),
  ('LEONARD', 1),
  ('You', 1),
  ('INT', 1)]}

# Section 4 - Named Entities

In [44]:
def process_dir(path):
    file_info = {}
    doc = {}
    

    for filename in os.listdir(path):
        if filename.endswith(".txt"):    
            file_path = os.path.join(path, filename)      
            with open(file_path, 'r', encoding="utf-8") as f:
                text = f.read()
                
               
                doc[filename]= nlp(text)
    return doc

In [45]:
file_entity_lists = {}
path = '/Users/anantbamba/Documents/Jupyter/data/A1'
doc = process_dir(path)
for filename in os.listdir(path):
    if filename.endswith(".txt"):
        file_entity_lists[filename] = []  # Create a new list for each filename
        for ent in doc[filename].ents:
            file_entity_lists[filename].append((ent.text, ent.label_))

In [46]:
file_dfs = {}

for filename in os.listdir(path):
    if filename.endswith(".txt"):    
        # create a df for the entities, from the list above
        file_dfs[filename] = pd.DataFrame(file_entity_lists[filename])
        # name the columns
        file_dfs[filename].columns = ['Entity', 'Label']

In [54]:
pd.set_option('display.max_rows', None)
print(file_dfs)

{'recipes.txt':                                      Entity        Label
0                                    Indian         NORP
1                                    Indian         NORP
2                                 every day         DATE
3                              A few months         DATE
4                                    Indian         NORP
5                                     Hindu         NORP
6                                    Indian         NORP
7                                  Turmeric       PERSON
8                                  Turmeric       PERSON
9                                    indian         NORP
10                                   Indian         NORP
11                                   1 hour         TIME
12                                       30     CARDINAL
13                                   1 hour         TIME
14                                       30     CARDINAL
15                                     Four     CARDINAL
16             

# Section 5 - Writeup

a. What do the most frequent words tell you about each of the 3 genres?

Ans) the most frequent words in all 3 files are generally commonly used grammatical words like the, is, and etc. But if we remove them, the difference in the three corpus becomes more clear. for the recipes, we start getting common ingredients like oil, chicken, salt etc, thus indicating the food related nature of the data. For the news article we start getting names like "oscars" and "film" that indicate the genre of this news to be related to the oscars . Lastly for the script, character names start appearing more as they are heavily featured in order to show dialogues. 

b. What do the named entities tell you about each of the 3 genres?

Ans) Firstly for recipe we get more quantitative and cardinal entities indicating how recipes are more likley to have numerical values and measurements for food preparation. Secondly for the new articles we start getting names of people and organizations, dates also become a bit more common as news articles mostly talk about celebrities and give references to past and future events, especially this one that is about the oscars. Lastly, the script is very heavy on people as entities mostly due to the fact that it may have alot of characters with dialogues, who may or may not be talking about each other.

c. Is the named entity output correct, in your opinion? If not, why not?

Ans) The names entity output is correct to some extent about 75% of the time. It may not be as correct because it cannot identify food items and just labels them as "person" and sometimes doesnt understand names in the oscars news and labels them wrong. It may be working on a limited or outdated dataset which may not give it 100% accuracy when it comes to current words.

d. How do you interpret the lexical diversity?

Ans) Lexical diversity refers to the amount of uniqueness in the text , the new article has higher diveristy as it is talking about alot of different movies and actors whereas the other texts recipes and script have comparatively lower diversity as recipes would usually feature recurring ingredients and scripts just have characters that would have dialogues and repeating scenarios.